In [3]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

path0= 'Q:\\SHA_KQ\\Digital Intelligence\\深度赋智-AI\\FTP\\hp_train\\2019 outbound\\'
path1= 'Q:\\SHA_KQ\\Digital Intelligence\\深度赋智-AI\\FTP\\hp_train\\2020 outbound\\'
path2= 'Q:\\SHA_KQ\\Digital Intelligence\\深度赋智-AI\\FTP\\hp_train\\2021 outbound\\'

In [6]:
%%time
dfs0 = pd.DataFrame()
for fname in os.listdir(path0):
    if re.search(r'\.xls$', fname):
        dfs0 = pd.concat(
            [dfs0 ,pd.read_excel(path0 + fname)], 
            axis = 0, 
            ignore_index = True)

dfs1 = pd.DataFrame()
for fname in os.listdir(path1):
    if re.search(r'\.xls$', fname):
        dfs1 = pd.concat(
            [dfs1 ,pd.read_excel(path1 + fname)], 
            axis = 0, 
            ignore_index = True)


dfs2 = pd.DataFrame()
for fname in os.listdir(path2):
    if re.search(r'\.xls$', fname):
        dfs2 = pd.concat(
            [dfs2 ,pd.read_excel(path2 + fname)], 
            axis = 0, 
            ignore_index = True)



In [ ]:
[dfs0.head(1),
dfs1.head(1),
dfs2.head(1)
]

In [ ]:
dfs3 = pd.DataFrame()
for fname in os.listdir(path2):
    if re.search(r'\.xlsx$', fname):
        dfs3 = pd.concat(
            [dfs3 ,pd.read_excel(path3 + fname)], 
            axis = 0, 
            ignore_index = True)